# Bayesian Neural Network Evaluation on MNIST
This notebook replicates the architecture and setup from the baseline paper and evaluates your custom Bayesian Neural Network implementation.

In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [2]:

# Import the BayesianNet from your file
from sbnn import BayesianNet, train

# Confirm import worked and structure is correct
model = BayesianNet(input_dim=784, hidden_dim=1000, output_dim=10).to(device)
print(model)


BayesianNet(
  (layer1): BayesianBinaryLinear()
  (layer2): BayesianBinaryLinear()
)


In [3]:

# MNIST loaders
batch_size = 256
transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])

train_loader = DataLoader(
    datasets.MNIST('./data', train=True, download=True, transform=transform),
    batch_size=batch_size, shuffle=True
)

test_loader = DataLoader(
    datasets.MNIST('./data', train=False, transform=transform),
    batch_size=batch_size, shuffle=False
)


100.0%
100.0%
100.0%
100.0%


In [4]:

# Initialize model
model = BayesianNet(input_dim=784, hidden_dim=1000, output_dim=10).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training function using your code
def train_mnist(model, train_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out, _, _ = model(x)
            loss = F.cross_entropy(out, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
        print(f"Epoch {epoch}: Loss={total_loss:.4f}, Accuracy={correct/total:.2%}")

train_mnist(model, train_loader, epochs=10)


Epoch 0: Loss=478.7492, Accuracy=42.18%
Epoch 1: Loss=438.3519, Accuracy=59.44%
Epoch 2: Loss=425.4933, Accuracy=64.94%
Epoch 3: Loss=409.6479, Accuracy=71.71%
Epoch 4: Loss=401.8973, Accuracy=75.02%
Epoch 5: Loss=397.7937, Accuracy=76.77%
Epoch 6: Loss=395.3733, Accuracy=77.80%
Epoch 7: Loss=393.6811, Accuracy=78.55%
Epoch 8: Loss=392.4736, Accuracy=79.05%
Epoch 9: Loss=390.6089, Accuracy=79.87%


In [5]:

# Evaluate with predictive uncertainty from Bayesian model
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        mean_preds, std_preds = model.predict_multiple(x, n_samples=20)
        preds = mean_preds.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(y.numpy())

from sklearn.metrics import accuracy_score
print("Test Accuracy:", accuracy_score(all_targets, all_preds))


Test Accuracy: 0.8299
